**Librerias y configuracion**

In [ ]:
import os
import re
import joblib
import unicodedata
from dotenv import load_dotenv
from telegram import Bot
from telegram.ext import ApplicationBuilder, MessageHandler, ContextTypes, filters
from telegram import Update
from logger import setup_logger

**Carga de modelo y vectoriazion**

In [ ]:
modelo = joblib.load("modelo/modelo_entrenado.pkl")
vectorizer = joblib.load("modelo/vectorizer.pkl")

**Configurar el logg de registros de errores y eventos**

In [ ]:
logger = setup_logger()

**Carga de variables de entorno**  

In [ ]:
load_dotenv()
telegram_bot_token = os.getenv("telegram_bot_token")

if not telegram_bot_token:
    raise ValueError("No se encontró el token en el archivo .env")

**Funciones para limpiar texto**

In [ ]:
#Limpia texto del dataset
def limpiar_texto(texto):
    texto = texto.lower()
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')
    texto = re.sub(r'[^\w\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

#Limpia las respuestas del usuario
def obtener_respuesta(pregunta_usuario):
    pregunta_limpia = limpiar_texto(pregunta_usuario)
    pregunta_vectorizada = vectorizer.transform([pregunta_limpia])
    respuesta = modelo.predict(pregunta_vectorizada)
    return respuesta[0]

**Procesar mensajes recibidos (Handler)**

In [ ]:
async def responder(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        pregunta = update.message.text
        if not pregunta or not pregunta.strip():
            await update.message.reply_text("Quizas quiciste decir decir algo o fue solo un suspiro😮‍💨.")
            return
        respuesta = obtener_respuesta(pregunta)
        await update.message.reply_text(f" {respuesta}")
    except Exception as e:
        logger.error(f"Error al responder: {e}")
        await update.message.reply_text("🖥️Si te digo que se cayo el sistema, me crees? jaja.")

**Creacion del bot, agregado del handler e inicializacion  en loop de escucha**

In [ ]:
async def main():
    app = ApplicationBuilder().token(telegram_bot_token).build()

    # Manejador para todos los mensajes de texto
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, responder))

    print("🤖 Bot en funcionamiento...")

    await app.initialize()
    await app.start()
    await app.updater.start_polling()


**Inicializacion y puesta en marcha del bot con comando para apagarlo**

In [ ]:
if __name__ == "__main__":
    import asyncio

    async def safe_main():
        await main()
        # Mantenemos el bot activo
        await asyncio.Event().wait()

    try:
        asyncio.get_event_loop().run_until_complete(safe_main())
    except KeyboardInterrupt:
        print("🔴 Bot detenido manualmente.")